# ROI Filter Playground

This notebook allows you to interactively tune parameters for identifying Regions of Interest (ROI) in your images.
It uses the same logic as `scripts/detect_roi.py` but exposes the parameters via sliders.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed
from pathlib import Path
import glob
import sys
import os

# Ensure we can import from scripts if needed, though we will copy logic here for editability
sys.path.append(os.path.abspath(".."))

%matplotlib inline

## 1. Core Logic

Here is the function that calculates the interest score. You can modify this cell to experiment with different filters.

In [2]:
def normalize01(x: np.ndarray) -> np.ndarray:
    """Normalize array to range [0, 1]."""
    x = x.astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    if mx - mn < 1e-6:
        return np.zeros_like(x, dtype=np.float32)
    return (x - mn) / (mx - mn)

def compute_roi_score(
    gray: np.ndarray,
    st_sigma: float = 2.0,
    w_grad: float = 0.5,
    w_aniso: float = 0.5,
    ksize_blur: int = 5,
) -> tuple:
    """
    Compute a score map for Regions of Interest.
    """
    # Normalize image to 0..1 float
    # Blur to reduce noise
    if ksize_blur % 2 == 0: ksize_blur += 1
    g = cv2.GaussianBlur(gray, (ksize_blur, ksize_blur), 0)
    
    # 1. Gradient Magnitude (Contrast)
    dx = cv2.Sobel(g, cv2.CV_32F, 1, 0, ksize=3)
    dy = cv2.Sobel(g, cv2.CV_32F, 0, 1, ksize=3)
    mag = cv2.magnitude(dx, dy)
    mag01 = normalize01(mag)

    # 2. Structure Tensor Anisotropy (Parallel Lines / Orientation stability)
    jxx = dx * dx
    jyy = dy * dy
    jxy = dx * dy
    
    # Smooth the tensor elements
    ksize = int(max(3, round(st_sigma * 6))) | 1
    jxx = cv2.GaussianBlur(jxx, (ksize, ksize), st_sigma)
    jyy = cv2.GaussianBlur(jyy, (ksize, ksize), st_sigma)
    jxy = cv2.GaussianBlur(jxy, (ksize, ksize), st_sigma)

    # Eigenvalues
    tr = jxx + jyy
    det = jxx * jyy - jxy * jxy
    disc = np.maximum(tr * tr - 4.0 * det, 0.0)
    root = np.sqrt(disc)
    
    l1 = 0.5 * (tr + root)
    l2 = 0.5 * (tr - root)
    
    # Anisotropy
    aniso = (l1 - l2) / (l1 + l2 + 1e-6)
    aniso01 = normalize01(aniso)

    # Combine
    score = (w_grad * mag01 + w_aniso * aniso01)
    score = normalize01(score)

    return score, mag01, aniso01

In [3]:
def extract_patches(
    img: np.ndarray,
    score_map: np.ndarray,
    patch_size: int = 128,
    max_patches: int = 50,
) -> list:
    """Simplified patch extraction returning boxes."""
    h, w = img.shape[:2]
    s_map = score_map.copy()
    patches = []
    half_size = patch_size // 2
    
    for _ in range(max_patches):
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(s_map)
        if max_val < 0.1: break
        cx, cy = max_loc
        
        x1 = max(0, cx - half_size)
        y1 = max(0, cy - half_size)
        x2 = min(w, x1 + patch_size)
        y2 = min(h, y1 + patch_size)
        
        if (x2 > x1) and (y2 > y1):
            patches.append(((x1, y1, x2, y2), max_val))
            
        nms_pad = int(patch_size * 0.8)
        n_x1 = max(0, cx - nms_pad // 2)
        n_y1 = max(0, cy - nms_pad // 2)
        n_x2 = min(w, cx + nms_pad // 2)
        n_y2 = min(h, cy + nms_pad // 2)
        s_map[n_y1:n_y2, n_x1:n_x2] = 0
        
    return patches

In [4]:
# Load file list
data_dir = Path("../data/raw")
image_files = sorted(list(data_dir.glob("*.png")) + list(data_dir.glob("*.jpg")) + list(data_dir.glob("*.jpeg")))
image_options = [(p.name, str(p)) for p in image_files]

if not image_files:
    print("No images found in data/raw/")
    image_options = [("No images", None)]

## 2. Interactive Visualization

In [5]:
def update_view(image_path, st_sigma, w_grad, w_aniso, ksize_blur, show_score_map, show_patches):
    if image_path is None:
        return
    
    img = cv2.imread(image_path)
    if img is None: return
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Compute
    score, mag, aniso = compute_roi_score(gray, st_sigma, w_grad, w_aniso, ksize_blur)
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(20, 8))
    
    # 1. Main View (Result)
    main_view = img_rgb.copy()
    if show_score_map:
        heatmap = cv2.applyColorMap((score * 255).astype(np.uint8), cv2.COLORMAP_JET)
        main_view = cv2.addWeighted(main_view, 0.6, cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB), 0.4, 0)
        
    if show_patches:
        patches = extract_patches(img, score, patch_size=128)
        for ((x1, y1, x2, y2), sc) in patches:
            cv2.rectangle(main_view, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
    axes[0].imshow(main_view)
    axes[0].set_title("Result (Patches / Score)")
    axes[0].axis('off')
    
    # 2. Gradient
    axes[1].imshow(mag, cmap='inferno')
    axes[1].set_title("Contrast (Gradient Magnitude)")
    axes[1].axis('off')
    
    # 3. Anisotropy
    axes[2].imshow(aniso, cmap='viridis')
    axes[2].set_title("Parallel Lines (Anisotropy)")
    axes[2].axis('off')
    
    plt.show()

interact(
    update_view,
    image_path=widgets.Dropdown(options=image_options, description='Image:'),
    st_sigma=widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='Sigma (Lines):'),
    w_grad=widgets.FloatSlider(value=0.4, min=0.0, max=2.0, step=0.1, description='W Contrast:'),
    w_aniso=widgets.FloatSlider(value=0.6, min=0.0, max=2.0, step=0.1, description='W Parallel:'),
    ksize_blur=widgets.IntSlider(value=5, min=1, max=15, step=2, description='Blur Size:'),
    show_score_map=widgets.Checkbox(value=False, description='Show Heatmap'),
    show_patches=widgets.Checkbox(value=True, description='Show Boxes')
);

interactive(children=(Dropdown(description='Image:', options=(('001.png', '../data/raw/001.png'), ('002.png', …